In [1]:
#import cobra
import cobra
import os

In [2]:
import os
import sys
MODELPATH_DIR = '/home/shahin/workspace/discngine_demo/discngine'
#BACK_DIR = os.path.join(MODELPATH_DIR,'backend')
sys.path.insert(0,MODELPATH_DIR)
os.environ.setdefault('DJANGO_SETTINGS_MODULE','discngine.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()
from django.db.models import Count, Q
import re
from discngine.settings import STATICFILES_DIRS

#### Importing models

In [3]:
from modelisation.models import GSModels, Genes, Metabolites, Reactions, Reaction_model, Reaction_products, Reaction_substrates

###### load bigg model with cobra

In [4]:
#loading model
model_paths = os.path.join(MODELPATH_DIR, 'static', 'models')
model = cobra.io.load_json_model(model_paths+'/iML1515.json')

In [5]:
model

Name,iML1515
Memory address,0x07f9d8db87f70
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


##### Adding the model to the database

In [10]:
iml1515 = GSModels(id=model.id, organism='E.Coli', objective=model.objective, nb_reactions=len(model.reactions), nb_metabolites=len(model.metabolites))
iml1515.save()

### adding all metabolites

In [6]:
for met in model.metabolites:
    metabolite = Metabolites(id=met.id, name=met.name, formula=met.formula, charge=met.charge, mol_weight=met.formula_weight)
    metabolite.save()
    
    # bigg ids
    bigg_ids = met.annotation.get('bigg.metabolite',None)
    metabolite.bigg = bigg_ids[0] if bigg_ids is not None else None
    # biocyc
    biocyc = met.annotation.get('biocyc', None)
    metabolite.biocyc = biocyc[0] if biocyc is not None else None
    # chebi
    chebi = met.annotation.get('chebi', None)
    metabolite.chebi = chebi[0] if chebi is not None else None
    # hmdb
    hmdb = met.annotation.get('hmdb', None)
    metabolite.hmdb = hmdb[0] if hmdb is not None else None
    # inchi_key
    inchi_key = met.annotation.get('inchi_key', None)
    metabolite.inchikey = inchi_key[0] if inchi_key is not None else None
    # kegg_compound
    kegg_compound = met.annotation.get('kegg.compound', None)
    metabolite.kegg_compound = kegg_compound[0] if kegg_compound is not None else None
    # kegg_drug
    kegg_drug = met.annotation.get('kegg.drug', None)
    metabolite.kegg_drug = kegg_drug[0] if kegg_drug is not None else None
    # metanetx.chemical
    mnxm = met.annotation.get('metanetx.chemical', None)
    metabolite.mnxm = mnxm[0] if mnxm is not None else None
    # sabiork
    sabiork = met.annotation.get('sabiork', None)
    metabolite.sabiork = sabiork[0] if sabiork is not None else None
    # sbo
    sbo = met.annotation.get('sbo', None)
    metabolite.sbo = sbo[0] if sbo is not None else None
    # seed.compound'
    seed = met.annotation.get('seed.compound', None)
    metabolite.seed = seed[0] if seed is not None else None
    
    metabolite.save()

/home/shahin/anaconda3/envs/prog/lib/python3.8/site-packages/cobra/core/metabolite.py:136: UserWarning: The element 'X' does not appear in the periodic table
  warn("The element %s does not appear in the periodic table" % e)
/home/shahin/anaconda3/envs/prog/lib/python3.8/site-packages/cobra/core/metabolite.py:136: UserWarning: The element 'R' does not appear in the periodic table
  warn("The element %s does not appear in the periodic table" % e)


###### adding smiles using selenium. Retrieving SMILES from biocyc pages for each metabolite ID

In [50]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

In [ ]:
driver = webdriver.Chrome("/home/shahin/workspace/discngine_demo/data/chromedriver")
for met in Metabolites.objects.filter(smiles=None):
    try:
        #url = 'identifiers.org/biocyc/'+met.biocyc
        #print(url)
        driver.get("https://biocyc.org/compound?orgid=META&id="+met.biocyc.split(':')[1])
        time.sleep(1)
        content = driver.page_source
        soup = BeautifulSoup(content)
        #print(soup)
        summary = soup.find('table', attrs={'class':"summaryTable3"})
        for a in summary.findAll('td', attrs={'class':'label'}):
            if a.text.strip() == 'SMILES':
                met.smiles = a.find_next().text.strip()
                #print(dir(a))
                #print(name)
                break
        met.save()
    except:pass
driver.close()    

##### generating MolFiles and 2D pics with RDKIT 

In [4]:
from rdkit import Chem
from rdkit.Chem import AllChem, Draw

In [40]:
for metabolite in Metabolites.objects.exclude(smiles=None):
    try:
        mol = Chem.MolFromSmiles(metabolite.smiles)
        filename = metabolite.id+'.png'
        img=Draw.MolToFile(mol, os.path.join(STATICFILES_DIRS[0], 'images','2D_molecule',filename))
    except:
        count+=1

RDKit ERROR: [18:41:34] SMILES Parse Error: syntax error while parsing: C(O)[C@H]1(O[C@@H]([C@H](O)[C@@H](O)[C@H](O[R1])1)O[C@H]4([C@H](O)[C@@H](O)[C@@H](O[C@H]2([C@H](O)[C@@H](O)[C@H](O[C@H](CO)2)O[C@H]3([C@H](O)[C@@H](O)[C@@H](O[R2])O[C@H](CO)3)))O[C@H](CO)4))
RDKit ERROR: [18:41:34] SMILES Parse Error: Failed parsing SMILES 'C(O)[C@H]1(O[C@@H]([C@H](O)[C@@H](O)[C@H](O[R1])1)O[C@H]4([C@H](O)[C@@H](O)[C@@H](O[C@H]2([C@H](O)[C@@H](O)[C@H](O[C@H](CO)2)O[C@H]3([C@H](O)[C@@H](O)[C@@H](O[R2])O[C@H](CO)3)))O[C@H](CO)4))' for input: 'C(O)[C@H]1(O[C@@H]([C@H](O)[C@@H](O)[C@H](O[R1])1)O[C@H]4([C@H](O)[C@@H](O)[C@@H](O[C@H]2([C@H](O)[C@@H](O)[C@H](O[C@H](CO)2)O[C@H]3([C@H](O)[C@@H](O)[C@@H](O[R2])O[C@H](CO)3)))O[C@H](CO)4))'
RDKit ERROR: [18:41:34] SMILES Parse Error: syntax error while parsing: C(O)[C@H]1(O[C@@H]([C@H](O)[C@@H](O)[C@H](O[R1])1)O[C@H]4([C@H](O)[C@@H](O)[C@@H](O[C@H]2([C@H](O)[C@@H](O)[C@H](O[C@H](CO)2)O[C@H]3([C@H](O)[C@@H](O)[C@@H](O[R2])O[C@H](CO)3)))O[C@H](CO)4))
RDKit ERROR

RDKit ERROR: [18:41:35] SMILES Parse Error: syntax error while parsing: CC(C)(COP(=O)([O-])OC[C@H](N[a
RDKit ERROR: [18:41:35] SMILES Parse Error: Failed parsing SMILES 'CC(C)(COP(=O)([O-])OC[C@H](N[a' for input: 'CC(C)(COP(=O)([O-])OC[C@H](N[a'
RDKit ERROR: [18:41:35] SMILES Parse Error: syntax error while parsing: CC(SCCNC(=O)CCNC([C@H](O)C(C)(C)COP(=O)([O-])OC[C@H](N[a
RDKit ERROR: [18:41:35] SMILES Parse Error: Failed parsing SMILES 'CC(SCCNC(=O)CCNC([C@H](O)C(C)(C)COP(=O)([O-])OC[C@H](N[a' for input: 'CC(SCCNC(=O)CCNC([C@H](O)C(C)(C)COP(=O)([O-])OC[C@H](N[a'
RDKit ERROR: [18:41:36] SMILES Parse Error: syntax error while parsing: CC(CC(=O)SCCNC(=O)CCNC([C@H](O)C(C)(C)COP(=O)([O-])OC[C@H](N[a
RDKit ERROR: [18:41:36] SMILES Parse Error: Failed parsing SMILES 'CC(CC(=O)SCCNC(=O)CCNC([C@H](O)C(C)(C)COP(=O)([O-])OC[C@H](N[a' for input: 'CC(CC(=O)SCCNC(=O)CCNC([C@H](O)C(C)(C)COP(=O)([O-])OC[C@H](N[a'
RDKit ERROR: [18:41:36] SMILES Parse Error: syntax error while parsing: C([R])(=O)C([O-]

RDKit WARNING: [18:41:41] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [18:41:41] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [18:41:41] WARNING: not removing hydrogen atom without neighbors
RDKit ERROR: [18:41:41] SMILES Parse Error: syntax error while parsing: C([R])[C@H](CC(=O)SCCNC(=O)CCNC([C@H](O)C(C)(C)COP(=O)([O-])OC[C@H](N[a
RDKit ERROR: [18:41:41] SMILES Parse Error: Failed parsing SMILES 'C([R])[C@H](CC(=O)SCCNC(=O)CCNC([C@H](O)C(C)(C)COP(=O)([O-])OC[C@H](N[a' for input: 'C([R])[C@H](CC(=O)SCCNC(=O)CCNC([C@H](O)C(C)(C)COP(=O)([O-])OC[C@H](N[a'
RDKit ERROR: [18:41:41] SMILES Parse Error: syntax error while parsing: COC(=O)CCC[C@@H](O)CC(=O)SCCNC(=O)CCNC([C@H](O)C(C)(C)COP(=O)([O-])OC[C@H](N[a
RDKit ERROR: [18:41:41] SMILES Parse Error: Failed parsing SMILES 'COC(=O)CCC[C@@H](O)CC(=O)SCCNC(=O)CCNC([C@H](O)C(C)(C)COP(=O)([O-])OC[C@H](N[a' for input: 'COC(=O)CCC[C@@H](O)CC(=O)SCCNC(=O)CCNC([C@H](O)C(C)(C)COP(=O)([O-])OC[C@H](N[a'
R

RDKit ERROR: [18:41:46] SMILES Parse Error: syntax error while parsing: C(OP([O-])(=O)O[a
RDKit ERROR: [18:41:46] SMILES Parse Error: Failed parsing SMILES 'C(OP([O-])(=O)O[a' for input: 'C(OP([O-])(=O)O[a'


72


###### adding PDB files from Mol

In [8]:
for met in Metabolites.objects.exclude(smiles=None):
    count=0
    try:
        mol = Chem.MolFromSmiles(met.smiles)
        ## adding hydrogenes and removes them to get 3D coordonates cf RDKIT
        m2 = Chem.AddHs(mol)
        AllChem.EmbedMolecule(m2,randomSeed=0xf00d)
        m2 = Chem.RemoveHs(m2)   
        molfile = met.id+'.mol'
        ## store coordonate to a molfile
        print(Chem.MolToMolBlock(m2),file=open(os.path.join(STATICFILES_DIRS[0], 'images','mol_files',molfile),'w+'))
        ## reading molfile as mol and generating a pdb file for 3Dmol.js
        mol3d = Chem.rdmolfiles.MolFromMolFile(os.path.join(STATICFILES_DIRS[0], 'images','mol_files',molfile))
        pdbfile = met.id+'.pdb'
        img=Chem.rdmolfiles.MolToPDBFile(mol3d, os.path.join(STATICFILES_DIRS[0], 'images','pdb_files',pdbfile))
    except:
        count+= 1
print(count, 'non réussi sur', len(Metabolites.objects.exclude(smiles=None)))

RDKit ERROR: [09:10:17] SMILES Parse Error: syntax error while parsing: C(O)[C@H]1(O[C@@H]([C@H](O)[C@@H](O)[C@H](O[R1])1)O[C@H]4([C@H](O)[C@@H](O)[C@@H](O[C@H]2([C@H](O)[C@@H](O)[C@H](O[C@H](CO)2)O[C@H]3([C@H](O)[C@@H](O)[C@@H](O[R2])O[C@H](CO)3)))O[C@H](CO)4))
RDKit ERROR: [09:10:17] SMILES Parse Error: Failed parsing SMILES 'C(O)[C@H]1(O[C@@H]([C@H](O)[C@@H](O)[C@H](O[R1])1)O[C@H]4([C@H](O)[C@@H](O)[C@@H](O[C@H]2([C@H](O)[C@@H](O)[C@H](O[C@H](CO)2)O[C@H]3([C@H](O)[C@@H](O)[C@@H](O[R2])O[C@H](CO)3)))O[C@H](CO)4))' for input: 'C(O)[C@H]1(O[C@@H]([C@H](O)[C@@H](O)[C@H](O[R1])1)O[C@H]4([C@H](O)[C@@H](O)[C@@H](O[C@H]2([C@H](O)[C@@H](O)[C@H](O[C@H](CO)2)O[C@H]3([C@H](O)[C@@H](O)[C@@H](O[R2])O[C@H](CO)3)))O[C@H](CO)4))'
RDKit ERROR: [09:10:17] SMILES Parse Error: syntax error while parsing: C(O)[C@H]1(O[C@@H]([C@H](O)[C@@H](O)[C@H](O[R1])1)O[C@H]4([C@H](O)[C@@H](O)[C@@H](O[C@H]2([C@H](O)[C@@H](O)[C@H](O[C@H](CO)2)O[C@H]3([C@H](O)[C@@H](O)[C@@H](O[R2])O[C@H](CO)3)))O[C@H](CO)4))
RDKit ERROR

RDKit ERROR: [09:10:43] UFFTYPER: Unrecognized atom type: Fe5 (0)
RDKit ERROR: [09:10:43] UFFTYPER: Unrecognized atom type: S_5+4 (1)
RDKit ERROR: [09:10:43] UFFTYPER: Unrecognized atom type: Fe5 (2)
RDKit ERROR: [09:10:43] UFFTYPER: Unrecognized atom type: S_5+4 (3)
RDKit ERROR: [09:10:43] UFFTYPER: Unrecognized atom type: Fe5 (4)
RDKit ERROR: [09:10:43] UFFTYPER: Unrecognized atom type: S_5+4 (5)
RDKit ERROR: [09:10:43] UFFTYPER: Unrecognized atom type: Fe5 (6)
RDKit ERROR: [09:10:43] UFFTYPER: Unrecognized atom type: S_5+4 (7)
RDKit ERROR: [09:10:44] SMILES Parse Error: syntax error while parsing: CC(C)(COP(=O)([O-])OC[C@H](N[a
RDKit ERROR: [09:10:44] SMILES Parse Error: Failed parsing SMILES 'CC(C)(COP(=O)([O-])OC[C@H](N[a' for input: 'CC(C)(COP(=O)([O-])OC[C@H](N[a'
RDKit ERROR: [09:10:45] SMILES Parse Error: syntax error while parsing: CC(SCCNC(=O)CCNC([C@H](O)C(C)(C)COP(=O)([O-])OC[C@H](N[a
RDKit ERROR: [09:10:45] SMILES Parse Error: Failed parsing SMILES 'CC(SCCNC(=O)CCNC([C@H]

RDKit ERROR: [10:28:34] UFFTYPER: Unrecognized hybridization for atom: 28
RDKit ERROR: [10:28:34] UFFTYPER: Unrecognized atom type: Fe (28)
RDKit ERROR: [10:28:35] UFFTYPER: Unrecognized hybridization for atom: 28
RDKit ERROR: [10:28:35] UFFTYPER: Unrecognized atom type: Fe (28)
RDKit ERROR: [10:28:36] UFFTYPER: Unrecognized atom type: Fe3 (5)
RDKit ERROR: [10:28:36] UFFTYPER: Unrecognized atom type: Fe3 (5)
RDKit ERROR: [10:28:37] UFFTYPER: Unrecognized atom type: Fe3 (5)
RDKit ERROR: [10:28:37] SMILES Parse Error: syntax error while parsing: C(OC(=O)C(CCCCCCCCCCCCCCCCCCCCCCCC)[C@H](O)CCCCCCCCCCCCCCCCCC1(C(CCCCCCCCCCCCCCCCC(OC)C(C)CCCCCCCCCCCCCCCCCC)C1))[C@@H]2([C@@H](O)[C@H](O)[C@H](O[a
RDKit ERROR: [10:28:37] SMILES Parse Error: Failed parsing SMILES 'C(OC(=O)C(CCCCCCCCCCCCCCCCCCCCCCCC)[C@H](O)CCCCCCCCCCCCCCCCCC1(C(CCCCCCCCCCCCCCCCC(OC)C(C)CCCCCCCCCCCCCCCCCC)C1))[C@@H]2([C@@H](O)[C@H](O)[C@H](O[a' for input: 'C(OC(=O)C(CCCCCCCCCCCCCCCCCCCCCCCC)[C@H](O)CCCCCCCCCCCCCCCCCC1(C(CCCCCCCCC

RDKit ERROR: [10:32:04] SMILES Parse Error: syntax error while parsing: C(CC(=O)[O-])(=O)SCCNC(=O)CCNC([C@H](O)C(C)(C)COP(=O)([O-])OC[C@H](N[a
RDKit ERROR: [10:32:04] SMILES Parse Error: Failed parsing SMILES 'C(CC(=O)[O-])(=O)SCCNC(=O)CCNC([C@H](O)C(C)(C)COP(=O)([O-])OC[C@H](N[a' for input: 'C(CC(=O)[O-])(=O)SCCNC(=O)CCNC([C@H](O)C(C)(C)COP(=O)([O-])OC[C@H](N[a'
RDKit ERROR: [10:32:19] SMILES Parse Error: syntax error while parsing: C([R2])(O)([R3])[R1]
RDKit ERROR: [10:32:19] SMILES Parse Error: Failed parsing SMILES 'C([R2])(O)([R3])[R1]' for input: 'C([R2])(O)([R3])[R1]'
RDKit ERROR: [10:32:19] SMILES Parse Error: syntax error while parsing: C([R2])(O)([R3])[R1]
RDKit ERROR: [10:32:19] SMILES Parse Error: Failed parsing SMILES 'C([R2])(O)([R3])[R1]' for input: 'C([R2])(O)([R3])[R1]'
RDKit ERROR: [10:32:19] SMILES Parse Error: syntax error while parsing: C([R2])(O)([R3])[R1]
RDKit ERROR: [10:32:19] SMILES Parse Error: Failed parsing SMILES 'C([R2])(O)([R3])[R1]' for input: 'C([R2])(

RDKit ERROR: [10:37:28] UFFTYPER: Unrecognized charge state for atom: 1
RDKit ERROR: [10:37:28] UFFTYPER: Unrecognized charge state for atom: 1
RDKit ERROR: [10:37:28] UFFTYPER: Unrecognized charge state for atom: 1
RDKit ERROR: [10:37:28] UFFTYPER: Unrecognized charge state for atom: 0
RDKit WARNING: [10:37:28] Warning: molecule is tagged as 3D, but all Z coords are zero
RDKit ERROR: [10:37:28] UFFTYPER: Unrecognized atom type: Se2+2 (4)
RDKit ERROR: [10:37:28] UFFTYPER: Unrecognized hybridization for atom: 14
RDKit ERROR: [10:37:28] UFFTYPER: Unrecognized atom type: Fe (14)
RDKit ERROR: [10:41:58] UFFTYPER: Unrecognized charge state for atom: 1
RDKit ERROR: [10:41:58] UFFTYPER: Unrecognized charge state for atom: 1
RDKit ERROR: [10:41:58] UFFTYPER: Unrecognized charge state for atom: 1
RDKit WARNING: [10:41:58] Warning: molecule is tagged as 3D, but all Z coords are zero
RDKit WARNING: [10:41:58] Warning: molecule is tagged as 3D, but all Z coords are zero
RDKit WARNING: [10:41:58] W

0 non réussi sur 1402


### adding all reactions in the database

In [21]:
for reac in model.reactions:
    #print(reac.id, reac.name, reac.subsystem, reac.notes, reac.annotation)
    print(reac.lower_bound)
    
    break

0.0


In [24]:
gsmodel = GSModels.objects.get(id=model.id)
for reac in model.reactions:
    reaction = Reactions(id=reac.id, name=reac.name, subsystem=reac.subsystem)
    reaction.save()
    
    # bigg ids
    bigg_ids = reac.annotation.get('bigg.reaction',None)
    reaction.bigg = bigg_ids[0] if bigg_ids is not None else None
    # biocyc
    biocyc = reac.annotation.get('biocyc', None)
    reaction.biocyc = biocyc[0] if biocyc is not None else None
    # hmdb
    hmdb = reac.annotation.get('hmdb', None)
    reaction.hmdb = hmdb[0] if hmdb is not None else None
    # ec_number
    ec_number = reac.annotation.get('ec-code', None)
    reaction.ec_number = ' '.join(ec_number) if ec_number is not None else None
    # kegg
    kegg = reac.annotation.get('kegg.reaction', None)
    reaction.kegg = kegg[0] if kegg is not None else None
    # metanetx.reaction
    mnxr = reac.annotation.get('metanetx.reaction', None)
    reaction.mnxr = mnxr[0] if mnxr is not None else None
    # rhea
    rhea = reac.annotation.get('rhea', None)
    reaction.rhea = rhea[0] if rhea is not None else None
    # sbo
    sbo = reac.annotation.get('sbo', None)
    reaction.sbo = sbo[0] if sbo is not None else None
    # seed.compound'
    seed = reac.annotation.get('seed.reaction', None)
    reaction.seed = seed[0] if seed is not None else None
    
    # Substrates
    for sub in reac.reactants:
        substrate = Metabolites.objects.get(id=sub.id)
        reaction.substrates.add(substrate, through_defaults={'stochiometry':reac.get_coefficient(sub.id)})
    
    # Products 
    for prod in reac.products:
        product = Metabolites.objects.get(id=prod.id)
        reaction.products.add(product, through_defaults={'stochiometry':reac.get_coefficient(prod.id)})
        
    # Model
    reaction.models.add(gsmodel, through_defaults={'lower_bound':reac.lower_bound, 'upper_bound':reac.upper_bound})
    
    reaction.save()

In [23]:
Reactions.objects.all().delete()

(16002,
 {'modelisation.Genes_reaction': 0,
  'modelisation.Reaction_substrates': 5248,
  'modelisation.Reaction_products': 5328,
  'modelisation.Reaction_model': 2712,
  'modelisation.Reactions': 2714})

In [11]:
GSModels.objects.all()

<QuerySet [<GSModels: iML1515>]>

In [8]:
GSModels.objects.all().delete()

(3,
 {'modelisation.GSModels_metabolites': 0,
  'modelisation.GSModels_genes': 0,
  'modelisation.Reaction_model': 1,
  'modelisation.GSModels': 2})

In [14]:
wd = os.path.dirname(os.getcwd())+'/'
dirmodels = wd+'static/metabolic/models_old/'
dirout = wd+'static/metabolic/models/'

In [9]:
os.listdir(dirmodels)

['iHN637.json',
 'RECON1.json',
 'iML1515.json',
 'iML1515_PKT_old.json',
 'iJO1366.json',
 'iML1515_PKT.json']

In [7]:
# os.system('cp /home/frek/Desktop/ibn_pathway.xlsx /media/frek/Storage/projects/chemocobra/ipynb')
data_ibn = pd.read_excel('ibn_pathway.xlsx',index_col=0,sheet_name=None)

In [8]:
data_ibn['reactions']

,name,lower_bound,upper_bound,gene_reaction_rule,reaction
id,,,,,
HMGCS,HMG-CoA Synthase,-1000,1000,S318,aacoa_c + accoa_c + h2o_c <==> hmgcoa_c + coa_...
ECH,HMG-CoA Dehydratase,-1000,1000,ECH,hmgcoa_c <==> mgcoa_c + h2o_c
AIBAB,MG-CoA Decarboxylase,0,1000,(aibA and aibB),mgcoa_c + h_c → mccoa_c + co2_c
TESB,Acyl-CoA Thioesterase,-1000,1000,b0452,mccoa_c + h2o_c → mca_c + coa_c
SS5,Prenate decarboxylase,0,1000,ss5,mca_c → ibn_c + co2_c
EX_IBN_e,IBN Exchange,0,1000,NaN,ibn_c →
ACACDCBX,Acetoacetate Decarboxylase,0,1000,NaN,acac_c + h_c --> acetone_c + co2_c
IPRNADH,Propane 2-Monooxygenase (EC 1.14.13.227),-1000,1000,NaN,acetone_c + h_c + nadh_c --> ipa_c + nad_c
IPRNADPH,Isopropanol Dehydrogenase (EC 1.1.1.80),-1000,1000,NaN,acetone_c + h_c + nadph_c --> ipa_c + nadp_c


In [11]:
[f for f in os.listdir(dirmodels) if 'old' not in f]

['iHN637.json',
 'RECON1.json',
 'iML1515.json',
 'iJO1366.json',
 'iML1515_PKT.json']

In [15]:
mets = {}
## Charge metabolites
for m_id in data_ibn['metabolites'].index:
    mets[m_id] = Metabolite(
        id = m_id,
        name = data_ibn['metabolites'].loc[m_id,'name'],
        formula = data_ibn['metabolites'].loc[m_id,'formula'],
        compartment = data_ibn['metabolites'].loc[m_id,'compartment']
    )
#     print(m_id,'charge:',mets[m_id].charge)
    
for filename in [f for f in os.listdir(dirmodels) if 'old' not in f]:
    model = cameo.load_model(dirmodels+filename)
    model.add_metabolites(mets.values())    
    ## add reactions
    for r_id in data_ibn['reactions'].index:
        r = Reaction(
            id = r_id,
            name = data_ibn['reactions'].loc[r_id,'name'],
            subsystem = 'IBN Pathway D',
            lower_bound = data_ibn['reactions'].loc[r_id,'lower_bound'],
            upper_bound = data_ibn['reactions'].loc[r_id,'upper_bound']
        )
        model.add_reaction(r)
        r.reaction = data_ibn['reactions'].loc[r_id,'reaction'].replace('→','-->')
        balanced = r.check_mass_balance()
    
    try:
        ACACCT = model.reactions.get_by_id('ACACCT')
    except:
        pass
    else:
        ACACCT.lower_bound = -1000
        print('ACACCT changed for',model.id)
            
    if 'iML1515' in model.id:
        model.objective = 'EX_acetone'
        res = model.optimize()
        print(model.id, res.objective_value)
    
    if model.id == 'iML1515_PKT_old':
        model.id = 'iML1515_PKT'
    cobra.io.save_json_model(model, dirout+filename)
    
    print(model.id, 'OK')
    

unknown metabolite 'acac_c' created
iHN637 OK
RECON1 OK
ACACCT changed for iML1515
iML1515 13.841159420289856
iML1515 OK
ACACCT changed for iJO1366
iJO1366 OK
ACACCT changed for iML1515_PKT_old
iML1515_PKT_old 14.792380952380945
iML1515_PKT OK


In [28]:

import requests

S = requests.Session()

URL = "https://en.wikipedia.org/w/api.php"

SEARCHPAGE = "isobutene"

PARAMS = {
    "action": "query",
    "format": "json",
    "list": "search",
    "srsearch": SEARCHPAGE
}

R = S.get(url=URL, params=PARAMS)
DATA = R.json()

if DATA['query']['search'][0]['title'] == SEARCHPAGE:
    print("Your search page '" + SEARCHPAGE + "' exists on English Wikipedia")

In [30]:
DATA['query']['search'][0]

{'ns': 0,
 'title': 'Isobutylene',
 'pageid': 1596651,
 'size': 5767,
 'wordcount': 424,
 'snippet': 'Drugs, and Biologicals (11th ed.), Merck, 1989, ISBN\xa0091191028X, 5024. <span class="searchmatch">Isobutene</span>, International Chemical Safety Card 1027, Geneva: International Programme',
 'timestamp': '2020-08-17T11:54:55Z'}

In [31]:
import wikipedia

In [32]:
print(wikipedia.summary("isobutene"))

Isobutylene (or 2-methylpropene) is a hydrocarbon with the formula (CH3)2C=CH2.  It is a four-carbon branched alkene (olefin), one of the four isomers of butylene.  A colourless flammable gas, it is of considerable industrial value.
